## Import Libraries and Declare API Key

In [1]:
import requests
import pandas as pd
import numpy as np
import signal
import time
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
API_KEY = {'X-API-key': 'RPCJTKLW'}

## Accesing the Past Heat History

### Historical Data Retrieval

In [3]:
pd_new = ''
with requests.Session() as s:
    s.headers.update(API_KEY)
    resp = s.get('http://localhost:9999/v1/news')
    if resp.ok:
        price_hist = resp.json()
        pd_list = pd.read_html(price_hist[0]['body'])
        pd_new = pd_list[0]
        pd_new = pd_new.iloc[: , 1:]
        pd_new.rename(columns=pd_new.iloc[0], inplace = True)
        pd_new.drop([0], inplace = True)
        pd_new.index = np.arange(0, len(pd_new))
        pd_new = pd_new.apply(pd.to_numeric, errors='ignore')
    else:
        print('API Error')

pd_new

,RSM1000,NGN,WHEL,GEAR
0,91.54,25.06,24.43,27.62
1,91.58,25.05,24.49,27.60
2,91.86,25.00,24.52,27.52
3,91.77,24.97,24.58,27.58
4,91.47,25.06,24.53,27.59
...,...,...,...,...
296,100.04,24.96,24.99,25.26
297,99.82,25.01,24.94,25.22
298,99.94,25.05,24.94,25.15
299,99.51,25.16,24.86,25.16


### Observed Historical Correlation

In [4]:
corr_df = pd_new.corr()
#reset symbol as index (rather than 0-X)
corr_df.head(10)
corr_df.style.set_caption("Observed Historical Correlation")

,RSM1000,NGN,WHEL,GEAR
RSM1000,1.000000,-0.485352,0.561287,-0.808386
NGN,-0.485352,1.000000,-0.214366,0.279847
WHEL,0.561287,-0.214366,1.000000,-0.621265
GEAR,-0.808386,0.279847,-0.621265,1.000000


### Observed Historical Volatillity

In [5]:
def volatility(tick):
    pd_new['Log returns'] = np.log(pd_new[tick]/pd_new[tick].shift())
    volatility = pd_new['Log returns'].std()*1200**.5
    return volatility

In [6]:
lst = [[volatility('RSM1000'), volatility('NGN'), volatility('WHEL'), volatility('GEAR')]]
df_vol = pd.DataFrame(lst, columns =['RSM1000', 'GEAR', 'WHEL', 'NGN'])
df_vol.style.set_caption("Observed Historical Volatillity")

,RSM1000,GEAR,WHEL,NGN
0,0.174748,0.136302,0.130900,0.156852


### Observed Historical Beta

In [7]:
def market_beta(tick):
    pd_new['Log returns_RSM'] = np.log(pd_new['RSM1000']/pd_new['RSM1000'].shift())
    pd_new['Log returns'] = np.log(pd_new[tick]/pd_new[tick].shift())
    
    beta = corr_df['RSM1000'][tick] * (pd_new['Log returns'].std() / pd_new['Log returns_RSM'].std())
    return beta  

In [8]:
lst = [[market_beta('GEAR'), market_beta('WHEL'), market_beta('NGN')]]
df_bet = pd.DataFrame(lst, columns =['GEAR', 'WHEL', 'NGN'])
df_bet.style.set_caption("Observed Historical Beta")

,GEAR,WHEL,NGN
0,-0.725598,0.420446,-0.378571


## Current Heat Trading Algorithm

### Basic Buy and Sell Functions

In [32]:
shutdown = False

class ApiException(Exception):
    pass

def signal_handler(signum, frame):
    global shutdown
    signal.signal(signal.SIGINT, signal.SIG_DFL)
    shutdown = True

def get_tick(session):
    resp = session.get('http://localhost:9999/v1/case')
    if resp.ok:
        case = resp.json()
        return case['tick']
    raise ApiException('Authorization error. Please check API key.')

def ticker_bid_ask(session, ticker):
    payload = {'ticker': ticker}
    resp = session.get('http://localhost:9999/v1/securities/book', params = payload)
    if resp.ok:
        book = resp.json()
        return book['bids'][0]['price'], book['asks'][0]['price']
    raise ApiException('Authorization error. Please check API key.')

def buy(session, ticker, type, q, price = None):
    if type == 'MARKET':
        resp = session.post('http://localhost:9999/v1/orders', params = {'ticker': ticker, 'type': 'MARKET', 'quantity': q, 'action': 'BUY'})
        if resp.ok:
            mkt_order = resp.json()
            id = mkt_order['order_id']
            print('The market buy order was submitted and has ID', id)
        else:
            print('The order was not successfully submitted!')
    else:
        resp = session.post('http://localhost:9999/v1/orders',
                            params={'ticker': ticker, 'type': 'LIMIT', 'quantity': q, 'action': 'BUY', 'price': price})
        if resp.ok:
            mkt_order = resp.json()
            id = mkt_order['order_id']
            print('The market buy order was submitted and has ID', id)
        else:
            print('The order was not successfully submitted!')

def sell(session, ticker, type, q, price = None):
    if type == 'MARKET':
        resp = session.post('http://localhost:9999/v1/orders', params = {'ticker': ticker, 'type': 'MARKET', 'quantity': q, 'action': 'SELL'})
        if resp.ok:
            mkt_order = resp.json()
            id = mkt_order['order_id']
            print('The market buy order was submitted and has ID', id)
        else:
            print('The order was not successfully submitted!')
    else:
        resp = session.post('http://localhost:9999/v1/orders',
                            params={'ticker': ticker, 'type': 'LIMIT', 'quantity': q, 'action': 'SELL', 'price': price})
        if resp.ok:
            mkt_order = resp.json()
            id = mkt_order['order_id']
            print('The market buy order was submitted and has ID', id)
        else:
            print('The order was not successfully submitted!')

def cancel(session, order_id):
    order_id = order_id # assuming the order to cancel has ID 100
    resp = session.delete('http://localhost:9999/v1/orders/{}'.format(order_id))
    if resp.ok:
        status = resp.json()
        success = status['success']
        print('The order was successfully cancelled?', success)
        
def get_limit(session):
    resp = session.get('http://localhost:9999/v1/limits')
    if resp.ok:
        lim = resp.json()
        return lim['gross'], lim['net'], lim['gross_limit'], lim['net_limit']
    else:
        print('API Error!')
        

### Divergence Functions

In [10]:
def init_price(session, ticker):
    resp = s.get('http://localhost:9999/v1/securities/history', params = {'ticker' : 'NGN'})
    if resp.ok:
        book = resp.json()
        return book[-1]['close']

def cur_price(session, ticker):
    resp = s.get('http://localhost:9999/v1/securities/history', params = {'ticker' : 'NGN'})
    if resp.ok:
        book = resp.json()
        return book[0]['close']
    
def ptd_ret(session, ticker):
    return cur_price(session, ticker)/init_price(session, ticker) - 1

def e_ptd_ret(session, ticker):
    return ptd_ret(session, 'RSM1000') * df_bet[ticker][0]

def div(session, ticker):
    return ptd_ret(session, ticker) - e_ptd_ret(session, ticker)

## Algorithm Execution Section

In [14]:
signal.signal(signal.SIGINT, signal_handler)

with requests.Session() as s:
    s.headers.update(API_KEY)
    while True:
        tick = get_tick(s)
        buy
        if div(s, 'NGN') > 0:
            buy(s, 'NGN','MARKET',  -10)
        if div(s, 'WHEL') > 0:
            buy(s, 'WHEL','MARKET', -10)
        if div(s, 'GEAR') > 0:
            buy(s, 'GEAR','MARKET', -10)
        if div(s, 'NGN') < 0:
            buy(s, 'NGN','MARKET', 10)
        if div(s, 'WHEL') < 0:
            buy(s, 'WHEL','MARKET', 10)
        if div(s, 'GEAR') < 0:
            buy(s, 'GEAR','MARKET', 10)
        time.sleep(1)

The market buy order was submitted and has ID 209
The market buy order was submitted and has ID 210
The market buy order was submitted and has ID 211
The market buy order was submitted and has ID 228
The market buy order was submitted and has ID 229
The market buy order was submitted and has ID 230
The market buy order was submitted and has ID 239
The market buy order was submitted and has ID 240
The market buy order was submitted and has ID 249
The market buy order was submitted and has ID 258
The market buy order was submitted and has ID 259
The market buy order was submitted and has ID 260
The market buy order was submitted and has ID 277
The market buy order was submitted and has ID 278
The market buy order was submitted and has ID 279
The market buy order was submitted and has ID 288
The market buy order was submitted and has ID 289
The market buy order was submitted and has ID 298
The market buy order was submitted and has ID 307
The market buy order was submitted and has ID 308


ConnectionError: HTTPConnectionPool(host='localhost', port=9999): Max retries exceeded with url: /v1/case (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000203CD02F370>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))